In [ ]:
from huggingface_hub import snapshot_download

PGN_DIR = "/root/sf_pgns"

snapshot_download(
    repo_id="official-stockfish/fishtest_pgns",
    repo_type="dataset",
    allow_patterns="25-01-1*/*/*.pgn.gz", 
    local_dir=PGN_DIR,
)


In [ ]:
import os
import re
import gzip
import math
from typing import Optional, Tuple, Iterator, List

import numpy as np
import chess
import chess.pgn
from tqdm import tqdm


# =========================================================
# CONSTANTS
# =========================================================

# Example Stockfish comments stored in node.comment:
#   "+1.20/26 3.2s"
#   "-1.18/26 2.8s"
#   "0.00/65 3.126s, Draw by adjudication"
#   "+M5/25 0.1s"
#
# We capture:
#   group(1): eval token (e.g. "-0.89" or "+M5" or "0.00")
#   group(2): depth        (e.g. "29")
#   group(3): time_s       (e.g. "23.772")
#

EVAL_RE = re.compile(
    r"([+#-]?(?:\d+(?:\.\d+)?|M\d+))/(\d+)\s+([0-9.]+)s"
)

# How aggressively to squash pawn evals into [-1, 1]
# value_normalized = clip(eval_pawns, [-VALUE_SCALE, VALUE_SCALE]) / VALUE_SCALE
VALUE_SCALE = 4.0  # 4 pawns = ±1.0; tune if needed


# =========================================================
# BOARD ENCODING: 18 PLANES
# =========================================================

def board_to_matrix(board: chess.Board) -> np.ndarray:
    """
    Convert a chess.Board to an 18x8x8 binary tensor.

    Planes 0-11: Piece planes
      0: White pawns     6: Black pawns
      1: White knights   7: Black knights
      2: White bishops   8: Black bishops
      3: White rooks     9: Black rooks
      4: White queens   10: Black queens
      5: White kings    11: Black kings

    Plane 12: Side to move (1 for white, 0 for black)
    Planes 13-16: Castling rights
      13: White kingside
      14: White queenside
      15: Black kingside
      16: Black queenside

    Plane 17: En passant target square (if any)
    """
    mat = np.zeros((18, 8, 8), dtype=np.float32)

    piece_mapping = {
        chess.PAWN:   0,
        chess.KNIGHT: 1,
        chess.BISHOP: 2,
        chess.ROOK:   3,
        chess.QUEEN:  4,
        chess.KING:   5,
    }

    for square, piece in board.piece_map().items():
        plane = piece_mapping[piece.piece_type]
        if piece.color == chess.BLACK:
            plane += 6
        row, col = divmod(square, 8)
        mat[plane, row, col] = 1.0

    # Side to move
    mat[12] = 1.0 if board.turn == chess.WHITE else 0.0

    # Castling rights
    mat[13] = 1.0 if board.has_kingside_castling_rights(chess.WHITE) else 0.0
    mat[14] = 1.0 if board.has_queenside_castling_rights(chess.WHITE) else 0.0
    mat[15] = 1.0 if board.has_kingside_castling_rights(chess.BLACK) else 0.0
    mat[16] = 1.0 if board.has_queenside_castling_rights(chess.BLACK) else 0.0

    # En passant square
    if board.ep_square is not None:
        ep_row, ep_col = divmod(board.ep_square, 8)
        mat[17, ep_row, ep_col] = 1.0

    return mat


def move_to_index(move: chess.Move) -> int:
    """
    Encode a move as an integer in range [0, 4096):
        from_square * 64 + to_square
    """
    return move.from_square * 64 + move.to_square


# =========================================================
# EVAL PARSING (RAW PAWNS)
# =========================================================

def parse_stockfish_eval(comment: str) -> Tuple[Optional[float], Optional[int], Optional[float], Optional[bool]]:
    """
    Parse a Stockfish comment like:
      '-0.23/19 2.7s'
      '+M5/25 0.1s'
      '0.00/65 3.126s, Draw by adjudication'

    Returns:
      score_pawns : score in pawns from the POV of the *player who just moved* (mover),
                    positive means advantage for mover. None if not found.
      depth       : search depth (int) or None
      time_s      : search time in seconds (float) or None
      is_mate     : True if mate score, False otherwise, None if no eval.
    """
    if not comment:
        return None, None, None, None

    m = EVAL_RE.search(comment)
    if not m:
        return None, None, None, None

    token  = m.group(1)   # "-0.23" or "+M5" or "0.00"
    depth  = int(m.group(2))
    time_s = float(m.group(3))

    # Mate token?
    if "M" in token:
        mate_m = re.match(r"([+-]?)M(\d+)", token)
        if not mate_m:
            return None, None, None, None
        sign = -1 if mate_m.group(1) == "-" else 1
        # Use a large pawn magnitude (e.g. 100 pawns)
        score_pawns = sign * 100.0
        is_mate = True
        return score_pawns, depth, time_s, is_mate

    # Normal score (pawns in the PGN)
    try:
        score_pawns = float(token)
    except ValueError:
        return None, None, None, None

    is_mate = False
    return score_pawns, depth, time_s, is_mate


# =========================================================
# VALUE NORMALIZATION (LINEAR, NO TANH)
# =========================================================

def normalize_eval_pawns(eval_pawns: float, scale: float = VALUE_SCALE) -> float:
    """
    Map a pawn eval (from POV of side to move) to [-1, +1]
    using simple linear scaling with clipping:

        clipped = clip(eval_pawns, -scale, +scale)
        v_norm  = clipped / scale

    Example (scale=4.0):
        eval_pawns = +1.0  -> +0.25
        eval_pawns = -2.0  -> -0.50
        eval_pawns = +10.0 -> +1.00 (saturated)
    """
    clipped = max(-scale, min(scale, eval_pawns))
    return clipped / scale


# =========================================================
# PGN PARSING (.pgn.gz, fishtest format)
# =========================================================

def parse_pgn_gz(
    filepath: str,
    max_games: Optional[int] = None,
    min_depth: int = 18,
) -> Iterator[Tuple[np.ndarray, np.ndarray, float]]:
    """
    Parse a gzip-compressed fishtest PGN file and yield training samples.

    For each *interior* position (not the final move of the game):

      - Input X      : board AFTER a move has been played.
      - y_policy     : next mainline move from that position (one-hot over 4096).
      - y_value_norm : **linearly normalized** eval in [-1, +1]
                       from POV of the side to move.

    Example snippet:
        e4 {+1.08/21 5.8s} e5 {-0.52/18 2.6s} 2. Be2 {+1.19/17 ...}

    In python-chess, node.comment holds just the inner text, e.g. "+1.08/21 5.8s".

    We emit:
      Row 1:
        X            = position after e4 (Black to move)
        raw_pawns    = -1.08   (flip because comment is from White/mover POV)
        y_value_norm = normalize_eval_pawns(raw_pawns, VALUE_SCALE)
        y_policy     = e5

      Row 2:
        X            = position after e5 (White to move)
        raw_pawns    = +0.52   (flip -0.52 from Black POV -> White POV)
        y_value_norm = normalize_eval_pawns(raw_pawns, VALUE_SCALE)
        y_policy     = Be2
    """
    with gzip.open(filepath, mode="rt", encoding="utf-8", errors="ignore") as f:
        game_counter = 0

        while True:
            game = chess.pgn.read_game(f)
            if game is None:
                break

            game_counter += 1
            if max_games is not None and game_counter > max_games:
                break

            board = game.board()
            node = game  # root node

            while node.variations:
                next_node = node.variation(0)
                move = next_node.move

                # Play the move; this is the position we want as input X
                board.push(move)

                # Parse engine eval on this move's comment
                raw_comment = next_node.comment or ""
                score_pawns, depth, time_s, is_mate = parse_stockfish_eval(raw_comment)

                # --- DEBUG: show first few comments for the first game ---
                if game_counter == 1:
                    if not hasattr(parse_pgn_gz, "_debug_shown"):
                        parse_pgn_gz._debug_shown = 0
                    if parse_pgn_gz._debug_shown < 10:
                        print("\n[DEBUG] Comment:", repr(raw_comment))
                        print("[DEBUG] Parsed eval:", score_pawns, "depth:", depth, "time:", time_s, "mate:", is_mate)
                        print("[DEBUG] Board FEN after move:", board.fen())
                        parse_pgn_gz._debug_shown += 1
                # -----------------------------------------------------------

                # If no eval or too shallow, skip this position
                if score_pawns is None or depth is None or depth < min_depth:
                    node = next_node
                    continue

                # score_pawns is from POV of the player who just moved.
                # In the resulting position, it's the *other* side to move.
                # We want eval from POV of the side to move => flip sign.
                y_value_pawns = -score_pawns

                # Linearly normalize pawn eval -> [-1, +1]
                y_value_norm = normalize_eval_pawns(y_value_pawns, VALUE_SCALE)

                # Policy target is the *next* mainline move if it exists
                if not next_node.variations:
                    # No next move from this position => no policy label; skip
                    node = next_node
                    continue

                next_move_node = next_node.variation(0)
                policy_move = next_move_node.move
                move_idx = move_to_index(policy_move)

                y_policy = np.zeros(4096, dtype=np.float32)
                y_policy[move_idx] = 1.0

                # Encode board AFTER 'move'
                X = board_to_matrix(board)

                # NOTE: we yield the **normalized** value
                yield X, y_policy, float(y_value_norm)

                # Advance the traversal
                node = next_node


# =========================================================
# BATCH SAVING
# =========================================================

def save_batch(
    X_list: List[np.ndarray],
    y_list_pol: List[np.ndarray],
    y_list_val: List[float],
    output_dir: str,
    batch_num: int,
) -> None:
    """
    Save a batch of data to disk as three .npy files:

      X_boards_batch_XXXX.npy : (B, 18, 8, 8)
      y_policy_batch_XXXX.npy : (B, 4096)
      y_value_batch_XXXX.npy  : (B,)  -- **normalized** evals in [-1, +1]
                                      from POV of side to move
    """
    X_arr     = np.stack(X_list, axis=0).astype(np.float32)
    y_arr_pol = np.stack(y_list_pol, axis=0).astype(np.float32)
    y_arr_val = np.array(y_list_val, dtype=np.float32)

    x_filename     = os.path.join(output_dir, f"X_boards_batch_{batch_num:04d}.npy")
    y_pol_filename = os.path.join(output_dir, f"y_policy_batch_{batch_num:04d}.npy")
    y_val_filename = os.path.join(output_dir, f"y_value_batch_{batch_num:04d}.npy")

    np.save(x_filename,     X_arr)
    np.save(y_pol_filename, y_arr_pol)
    np.save(y_val_filename, y_arr_val)

    x_size     = os.path.getsize(x_filename)     / (1024**2)
    y_pol_size = os.path.getsize(y_pol_filename) / (1024**2)
    y_val_size = os.path.getsize(y_val_filename) / (1024**2)

    print(
        f"    Batch {batch_num:04d}: {len(X_list):,} samples "
        f"({x_size:.1f}MB + {y_pol_size:.1f}MB + {y_val_size:.1f}MB)"
    )


# =========================================================
# PGN FILE DISCOVERY
# =========================================================

def collect_pgn_gz_files(root_dir: str) -> List[str]:
    """
    Recursively collect all .pgn.gz files under root_dir.
    """
    paths: List[str] = []
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for fname in filenames:
            if fname.endswith(".pgn.gz"):
                paths.append(os.path.join(dirpath, fname))
    paths.sort()
    return paths


# =========================================================
# TOP-LEVEL DRIVER
# =========================================================

def process_all_pgns(
    pgn_root: str,
    output_dir: str,
    max_positions: Optional[int] = None,
    batch_size: int = 10_000,
    max_games_per_file: Optional[int] = None,
    min_depth: int = 18,
) -> None:
    """
    Process all .pgn.gz files under pgn_root and save in batches.

    Each emitted row satisfies:
      - X encodes the position AFTER a move (side to move = board.turn).
      - y_policy is the next mainline move from that position.
      - y_value is the **linearly normalized** engine eval (in [-1, +1])
        from POV of side to move.
    """
    os.makedirs(output_dir, exist_ok=True)

    pgn_files = collect_pgn_gz_files(pgn_root)

    print(f"Starting processing of {len(pgn_files)} PGN files under {pgn_root}")
    print(f"Batch size: {batch_size:,} positions per batch")
    if max_positions is not None:
        print(f"Global position cap: {max_positions:,}")
    if max_games_per_file is not None:
        print(f"Per-file game cap: {max_games_per_file}")
    print(f"Min eval depth: {min_depth}")
    print(f"Output directory: {output_dir}")
    print(f"VALUE_SCALE: {VALUE_SCALE} (v_norm = clip(pawns, ±VALUE_SCALE) / VALUE_SCALE)")
    print("-" * 60)

    all_X: List[np.ndarray] = []
    all_y_pol: List[np.ndarray] = []
    all_y_val: List[float] = []

    total_positions = 0
    batch_count = 0

    # Running stats over normalized values
    val_count = 0
    val_sum = 0.0
    val_sq_sum = 0.0
    val_min = float("inf")
    val_max = float("-inf")

    for i, pgn_path in enumerate(pgn_files):
        if max_positions is not None and total_positions >= max_positions:
            print(f"Reached global position cap of {max_positions:,}")
            break

        print(f"📁 Processing file {i+1}/{len(pgn_files)}: {pgn_path}")

        file_pos_count = 0

        try:
            iterator = parse_pgn_gz(
                filepath=pgn_path,
                max_games=max_games_per_file,
                min_depth=min_depth,
            )

            for X, y_pol, y_val_norm in tqdm(iterator, desc=f"File {i+1}/{len(pgn_files)}", unit="pos"):
                all_X.append(X)
                all_y_pol.append(y_pol)
                all_y_val.append(y_val_norm)

                # Update stats (normalized values in [-1, 1])
                val_count += 1
                val_sum += y_val_norm
                val_sq_sum += y_val_norm * y_val_norm
                val_min = min(val_min, y_val_norm)
                val_max = max(val_max, y_val_norm)

                file_pos_count += 1
                total_positions += 1

                if len(all_X) >= batch_size:
                    save_batch(all_X, all_y_pol, all_y_val, output_dir, batch_count)
                    all_X, all_y_pol, all_y_val = [], [], []
                    batch_count += 1
                    print(
                        f"Saved batch {batch_count-1} | "
                        f"Running total: {total_positions:,} positions"
                    )

                if max_positions is not None and total_positions >= max_positions:
                    print(f"\n Reached global position cap of {max_positions:,}")
                    break

        except Exception as e:
            print(f"Error processing {pgn_path}: {e}")
            continue

        print(f"Parsed positions from file: {file_pos_count:,}")
        print(f"Running total: {total_positions:,} positions")

        if file_pos_count == 0:
            print("WARNING: 0 samples from this file. Likely no eval comments passed the filters.")

        if max_positions is not None and total_positions >= max_positions:
            break

    # Save remaining
    if all_X:
        save_batch(all_X, all_y_pol, all_y_val, output_dir, batch_count)
        print(f"Saved final batch {batch_count}")

    print("-" * 60)
    print(f"COMPLETED! Processed {total_positions:,} positions total")

    # Value distribution (normalized)
    if val_count > 0:
        mean = val_sum / val_count
        var = max(val_sq_sum / val_count - mean * mean, 0.0)
        std = math.sqrt(var)

        print("Normalized value distribution (over all emitted positions):")
        print(f"   Count: {val_count:,}")
        print(f"   Min:   {val_min:.3f}")
        print(f"   Max:   {val_max:.3f}")
        print(f"   Mean:  {mean:.3f}")
        print(f"   Std:   {std:.3f}")

    # Rough board tensor size estimate
    print(f"Approx board tensor size: ~{(total_positions * 18 * 8 * 8) / (1024**2):.1f} MB")


# =========================================================
# USAGE EXAMPLE (script entry point)
# =========================================================


In [ ]:

PGN_ROOT   = "/root/sf_pgns"                    
OUTPUT_DIR = "./processed_data_playerpov_norm"  

process_all_pgns(
    pgn_root=PGN_ROOT,
    output_dir=OUTPUT_DIR,
    max_positions=5_000_000,   
    batch_size=10_000,
    max_games_per_file=None,
    min_depth=18,
)

print("\nData ready for training!")
print("Output per batch:")
print("   X_boards_batch_XXXX.npy : (B, 18, 8, 8)")
print("   y_policy_batch_XXXX.npy : (B, 4096)")
print("   y_value_batch_XXXX.npy  : (B,)  -- normalized evals in [-1, +1] (POV = side to move)")